# In this project we will Create a Live Object Detection Program

In [64]:
import cv2
import numpy as np

In [65]:
labels_path = "../yolo/coco.names"
config_path = "../yolo/yolov3.cfg" 
weights_path = "../yolo/yolov3.weights"


labels = open(labels_path).read().split()
network = cv2.dnn.readNetFromDarknet(config_path,weights_path)
ln = network.getUnconnectedOutLayersNames()

In [66]:
def yolo_predection(img):
    Boxes = []
    Confidences = []
    Labels = []
    W,H = img.shape[:2]
    pre_img = cv2.dnn.blobFromImage(img,1/255.0,(416,416),swapRB=True)
    network.setInput(pre_img)
    outputs = network.forward(ln)
    for layer in outputs:
        for detected in layer:
            score = detected[5:]
            classId = np.argmax(score)
            confidence = score[classId]
            if float(confidence)>0.8:
                Object = labels[classId]
                CenterX,CenterY,Width,Height = (detected[:4]*np.array([W,H,W,H])).astype("int")
                X,Y = int(CenterX - Width/2),int(CenterY - Height/2)
                Boxes.append([X,Y,int(Width),int(Height)])
                Confidences.append(float(confidence))
                Labels.append(Object)
    best_output = cv2.dnn.NMSBoxes(Boxes,Confidences,0.5,0.4)
    for i in range(len(Boxes)):
        if i in best_output:
            my_lab = Labels[i]
            X,Y,W,H = Boxes[i]
            cv2.rectangle(img,(X,Y),(X+W,Y+H),(0,0,255),2)
            cv2.putText(img,my_lab,(X,Y+10),cv2.FONT_HERSHEY_SIMPLEX,.3,(0,255,0),1)
    return img
    
            
                
                

In [58]:
cam = cv2.VideoCapture(0)
while True:
    temp,img = cam.read()
    if temp:
        pred_img = yolo_predection(img)
        cv2.imshow("working",pred_img)
        if cv2.waitKey(1)==13:
            break
cam.release()
cv2.destroyAllWindows()